<a href="https://colab.research.google.com/github/VijayNale/incident_project/blob/master/Model_Incident2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files

In [2]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv("/content/drive/My Drive/Incident/kaggle/train.csv")
test = pd.read_csv("/content/drive/My Drive/Incident/kaggle/test.csv")

In [0]:
train.drop(['Unnamed: 0','updated_at_d'], axis=1 , inplace =True)
test.drop(['Unnamed: 0','updated_at_d'], axis=1 , inplace =True)

In [8]:
train

,ID,ID_status,active,count_reassign,count_opening,count_updated,ID_caller,opened_by,Created_by,updated_by,type_contact,location,category_ID,user_symptom,Support_group,support_incharge,Doc_knowledge,confirmation_check,impact,notify,opened_time_d,created_at_d
0,0.0,6.0,1.0,0.0,0.0,0.0,1455.0,200.0,151.0,94.0,3.0,43.0,44.0,491.0,46.0,131.0,1.0,0.0,1.0,0.0,29.0,29.0
1,0.0,7.0,1.0,0.0,0.0,3.0,1455.0,200.0,151.0,585.0,3.0,43.0,44.0,491.0,46.0,96.0,1.0,0.0,1.0,0.0,29.0,29.0
2,0.0,5.0,0.0,0.0,0.0,4.0,1455.0,200.0,151.0,681.0,3.0,43.0,44.0,491.0,46.0,109.0,1.0,0.0,1.0,0.0,29.0,29.0
3,1.0,0.0,1.0,1.0,0.0,1.0,1455.0,120.0,58.0,94.0,3.0,62.0,29.0,344.0,14.0,164.0,1.0,0.0,1.0,0.0,29.0,29.0
4,1.0,0.0,1.0,1.0,0.0,2.0,1455.0,120.0,58.0,94.0,3.0,62.0,29.0,344.0,14.0,164.0,1.0,0.0,1.0,0.0,29.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99193,24567.0,7.0,1.0,1.0,0.0,3.0,165.0,9.0,161.0,145.0,1.0,104.0,31.0,360.0,22.0,0.0,0.0,1.0,1.0,0.0,16.0,1.0
99194,24567.0,5.0,0.0,1.0,0.0,4.0,165.0,9.0,161.0,145.0,1.0,104.0,31.0,360.0,22.0,0.0,0.0,1.0,1.0,0.0,16.0,1.0
99195,24568.0,0.0,1.0,0.0,0.0,0.0,165.0,9.0,159.0,681.0,1.0,104.0,31.0,360.0,62.0,0.0,0.0,0.0,1.0,0.0,16.0,1.0
99196,24568.0,0.0,1.0,1.0,0.0,1.0,165.0,9.0,161.0,416.0,1.0,104.0,31.0,360.0,22.0,71.0,0.0,0.0,1.0,0.0,16.0,1.0


In [0]:
test

,ID,ID_status,active,count_reassign,count_opening,count_updated,ID_caller,opened_by,Created_by,updated_by,type_contact,location,category_ID,user_symptom,Support_group,support_incharge,Doc_knowledge,confirmation_check,notify,opened_time_d,created_at_d
0,0.0,7.0,1.0,0.0,0.0,2.0,1361.0,190.0,141.0,312.0,3.0,40.0,44.0,446.0,45.0,127.0,1.0,0.0,0.0,29.0,29.0
1,1.0,6.0,1.0,0.0,0.0,0.0,1361.0,114.0,55.0,379.0,3.0,58.0,29.0,312.0,61.0,213.0,1.0,0.0,0.0,29.0,29.0
2,1.0,0.0,1.0,1.0,0.0,3.0,1361.0,114.0,55.0,412.0,3.0,58.0,29.0,312.0,14.0,161.0,1.0,0.0,0.0,29.0,29.0
3,1.0,7.0,1.0,1.0,0.0,7.0,1361.0,114.0,55.0,140.0,3.0,58.0,29.0,312.0,14.0,213.0,1.0,0.0,0.0,29.0,29.0
4,2.0,6.0,1.0,0.0,0.0,4.0,3279.0,190.0,141.0,65.0,3.0,98.0,9.0,312.0,61.0,87.0,1.0,0.0,0.0,29.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42509,19754.0,5.0,0.0,0.0,0.0,1.0,1361.0,160.0,97.0,487.0,2.0,142.0,39.0,368.0,61.0,107.0,0.0,0.0,0.0,13.0,1.0
42510,19755.0,6.0,1.0,0.0,0.0,0.0,4681.0,160.0,98.0,472.0,2.0,142.0,39.0,368.0,61.0,112.0,0.0,0.0,0.0,14.0,1.0
42511,19756.0,6.0,1.0,0.0,0.0,0.0,838.0,160.0,97.0,472.0,2.0,142.0,39.0,368.0,61.0,108.0,0.0,0.0,0.0,15.0,1.0
42512,19757.0,6.0,1.0,0.0,0.0,0.0,867.0,160.0,97.0,472.0,2.0,142.0,39.0,368.0,61.0,108.0,0.0,0.0,0.0,15.0,1.0


In [0]:
## Get the Fraud and the normal dataset 
X = train.drop('impact', axis=1)
Y = train['impact']

In [0]:
from imblearn.over_sampling import SMOTE
smote=SMOTE(sampling_strategy= 'all')
x,y=smote.fit_sample(X,Y)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/loca

In [0]:
from collections import Counter
print('Original dataset shape {}'.format(Counter(train['impact'])))
print('Resampled dataset shape {}'.format(Counter(y)))  #second class size is 50% of first class

Original dataset shape Counter({1.0: 94034, 2.0: 2720, 0.0: 2444})
Resampled dataset shape Counter({1.0: 94034, 0.0: 94034, 2.0: 94034})


In [0]:
type(x)

numpy.ndarray

In [0]:
x = pd.DataFrame(x)
y = pd.DataFrame(y)

Train = pd.concat([x,y], axis=1)

In [0]:
Train.columns = train.columns

In [0]:
Train.isnull().sum()

ID                    0
ID_status             0
active                0
count_reassign        0
count_opening         0
count_updated         0
ID_caller             0
opened_by             0
Created_by            0
updated_by            0
type_contact          0
location              0
category_ID           0
user_symptom          0
Support_group         0
support_incharge      0
Doc_knowledge         0
confirmation_check    0
impact                0
notify                0
opened_time_d         0
created_at_d          0
dtype: int64

In [0]:
from sklearn.model_selection import StratifiedShuffleSplit as sss
split = sss(n_splits = 5, test_size = 0.2, random_state = 42)
for train_index , test_index in split.split(Train.drop('impact',axis=1), Train['impact']):
    train_x = Train.loc[train_index]
    train_y = Train.loc[test_index]

In [0]:
train_x

,ID,ID_status,active,count_reassign,count_opening,count_updated,ID_caller,opened_by,Created_by,updated_by,type_contact,location,category_ID,user_symptom,Support_group,support_incharge,Doc_knowledge,confirmation_check,impact,notify,opened_time_d,created_at_d
132619,5096.0,6.000000,1.0,0.342669,0.0,7.342669,1042.0,206.0,135.0,91.258648,3.0,104.0,46.0,126.000000,52.915979,122.915979,0.0,0.000000,0.0,17.0,1.0,0.0
15466,3056.0,0.000000,1.0,0.000000,0.0,0.000000,4226.0,166.0,104.0,697.000000,3.0,123.0,35.0,357.000000,11.000000,139.000000,1.0,0.000000,0.0,3.0,3.0,1.0
89640,22029.0,6.000000,1.0,1.000000,0.0,1.000000,1389.0,188.0,124.0,499.000000,3.0,217.0,35.0,353.000000,62.000000,137.000000,0.0,0.000000,0.0,19.0,1.0,1.0
96133,23800.0,2.000000,1.0,4.000000,0.0,18.000000,2717.0,31.0,1.0,419.000000,3.0,7.0,26.0,209.000000,51.000000,54.000000,0.0,0.000000,0.0,30.0,30.0,1.0
70709,16866.0,3.000000,1.0,2.000000,0.0,5.000000,3408.0,59.0,6.0,636.000000,3.0,217.0,31.0,379.000000,47.000000,120.000000,0.0,0.000000,0.0,30.0,30.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135061,20716.0,2.627045,1.0,0.000000,0.0,1.686478,3665.0,120.0,58.0,396.773730,3.0,59.0,35.0,267.940567,28.000000,56.000000,0.0,0.000000,0.0,16.0,16.0,0.0
147386,6949.0,2.043140,1.0,0.659477,0.0,4.318953,3233.0,185.0,122.0,543.000000,3.0,183.0,26.0,17.000000,50.788896,215.000000,0.0,0.000000,0.0,23.0,23.0,0.0
53674,12223.0,5.000000,0.0,0.000000,0.0,3.000000,1856.0,31.0,1.0,681.000000,3.0,77.0,15.0,357.000000,62.000000,31.000000,0.0,1.000000,0.0,4.0,1.0,1.0
87990,21572.0,0.000000,1.0,0.000000,0.0,0.000000,2524.0,96.0,34.0,430.000000,3.0,133.0,46.0,247.000000,26.000000,69.000000,0.0,0.000000,0.0,18.0,18.0,1.0


In [0]:
y = train_x['impact']
X = train_x.drop(['impact'], axis = 1)

y_t = train_y['impact']
X_t = train_y.drop(['impact'], axis = 1)

In [0]:
X_t

,ID,ID_status,active,count_reassign,count_opening,count_updated,ID_caller,opened_by,Created_by,updated_by,type_contact,location,category_ID,user_symptom,Support_group,support_incharge,Doc_knowledge,confirmation_check,notify,opened_time_d,created_at_d
191691,21360.398832,0.000000,1.000000,0.022265,0.0,0.044531,1332.874195,14.000000,154.000000,679.708602,3.0,43.356248,31.244920,404.647646,60.931257,203.129699,0.0,0.000000,17.000000,1.356248,2.0
28505,5849.000000,6.000000,1.000000,0.000000,0.0,3.000000,3142.000000,139.000000,77.000000,525.000000,3.0,7.000000,35.000000,72.000000,45.000000,58.000000,0.0,0.000000,21.000000,21.000000,0.0
61827,14352.000000,7.000000,1.000000,3.000000,0.0,6.000000,4012.000000,60.000000,23.000000,632.000000,3.0,104.000000,32.000000,357.000000,43.000000,116.000000,0.0,1.000000,20.000000,20.000000,0.0
65424,15388.000000,0.000000,1.000000,1.000000,0.0,1.000000,1320.000000,185.000000,122.000000,24.000000,3.0,217.000000,42.000000,357.000000,27.000000,202.000000,0.0,0.000000,25.000000,25.000000,1.0
64460,15105.000000,7.000000,1.000000,1.000000,0.0,4.000000,880.000000,60.000000,23.000000,97.000000,3.0,104.000000,12.000000,135.000000,15.000000,191.000000,0.0,1.000000,25.000000,25.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77727,18728.000000,5.000000,0.000000,0.000000,0.0,1.000000,3910.000000,4.000000,143.000000,681.000000,3.0,104.000000,46.000000,403.000000,62.000000,190.000000,0.0,0.000000,5.000000,1.000000,1.0
193510,18130.130462,0.000000,1.000000,1.000000,0.0,1.000000,1968.000000,46.000000,16.000000,362.000000,3.0,87.000000,17.000000,357.000000,55.000000,52.000000,0.0,0.000000,5.000000,5.000000,2.0
200050,23506.000000,2.062881,0.587424,0.000000,0.0,0.825153,5191.000000,14.000000,154.000000,681.000000,3.0,179.000000,31.000000,377.860440,62.000000,205.000000,0.0,0.412576,27.000000,1.000000,2.0
145002,18553.000000,6.000000,1.000000,0.000000,0.0,2.311538,4845.000000,206.000000,135.000000,435.384601,3.0,59.000000,35.000000,355.103846,28.000000,56.000000,0.0,0.000000,5.000000,5.000000,0.0


In [0]:
from sklearn.ensemble import RandomForestClassifier
RFmodel = RandomForestClassifier(n_jobs = 2, oob_score=True , n_estimators = 100, criterion = 'entropy')
RFmodel.fit(X , y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [0]:
  X_t['pred'] = RFmodel.predict(X_t)

  from sklearn.metrics import confusion_matrix, recall_score, precision_score , f1_score  , classification_report , accuracy_score
  def get_metrics(y_test, y_predicted):
      accuracy = accuracy_score(y_test, y_predicted)
      precision = precision_score(y_test, y_predicted, average='weighted')
      recall = recall_score(y_test, y_predicted, average='weighted')
      f1 = f1_score(y_test, y_predicted, average='weighted')
      return accuracy, precision, recall, f1
  accuracy, precision, recall, f1 = get_metrics(y_t , X_t['pred'])
  print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))  #0.981

accuracy = 1.000 
precision = 1.000 
recall = 1.000 
f1 = 1.000


In [0]:
print(confusion_matrix(y_t , X_t['pred']))
print(classification_report(y_t , X_t['pred']))  #98

[[56404     0]
 [    0    17]]
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     56404
         1.0       1.00      1.00      1.00        17

    accuracy                           1.00     56421
   macro avg       1.00      1.00      1.00     56421
weighted avg       1.00      1.00      1.00     56421



In [0]:
y = Train['impact']
X = Train.drop(['impact'], axis = 1)

In [0]:
X

,ID,ID_status,active,count_reassign,count_opening,count_updated,ID_caller,opened_by,Created_by,updated_by,type_contact,location,category_ID,user_symptom,Support_group,support_incharge,Doc_knowledge,confirmation_check,notify,opened_time_d,created_at_d
0,0.000000,6.000000,1.000000,0.000000,0.0,0.000000,1455.000000,200.000000,151.000000,94.000000,3.0,43.000000,44.000000,491.000000,46.000000,131.000000,1.0,0.000000,29.0,29.000000,1.0
1,0.000000,7.000000,1.000000,0.000000,0.0,3.000000,1455.000000,200.000000,151.000000,585.000000,3.0,43.000000,44.000000,491.000000,46.000000,96.000000,1.0,0.000000,29.0,29.000000,1.0
2,0.000000,5.000000,0.000000,0.000000,0.0,4.000000,1455.000000,200.000000,151.000000,681.000000,3.0,43.000000,44.000000,491.000000,46.000000,109.000000,1.0,0.000000,29.0,29.000000,1.0
3,1.000000,0.000000,1.000000,1.000000,0.0,1.000000,1455.000000,120.000000,58.000000,94.000000,3.0,62.000000,29.000000,344.000000,14.000000,164.000000,1.0,0.000000,29.0,29.000000,1.0
4,1.000000,0.000000,1.000000,1.000000,0.0,2.000000,1455.000000,120.000000,58.000000,94.000000,3.0,62.000000,29.000000,344.000000,14.000000,164.000000,1.0,0.000000,29.0,29.000000,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282097,1767.000000,0.000000,1.000000,0.000000,0.0,2.054564,3703.000000,14.000000,154.000000,655.000000,3.0,7.000000,23.000000,2.000000,17.000000,73.000000,1.0,0.000000,3.0,3.000000,2.0
282098,1181.028573,5.330837,0.330837,0.669163,0.0,58.217144,2545.458129,118.827782,56.166108,681.000000,3.0,100.565712,40.691626,188.579310,36.586798,89.151132,1.0,0.669163,3.0,2.338325,2.0
282099,17769.988072,5.000000,0.000000,0.000000,0.0,1.000000,1968.000000,46.000000,16.000000,681.000000,3.0,87.000000,17.000000,357.000000,62.000000,52.000000,0.0,0.000000,5.0,1.000000,2.0
282100,322.000000,6.000000,1.000000,2.427155,0.0,4.337338,4849.000000,88.000000,20.000000,454.303394,3.0,43.000000,37.021930,285.528985,50.854309,69.000000,1.0,0.000000,3.0,3.000000,2.0


In [0]:
from sklearn.ensemble import RandomForestClassifier
RFmodel = RandomForestClassifier(n_jobs = 2, oob_score=True , n_estimators = 100, criterion = 'entropy')
RFmodel.fit(X , y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=2,
                       oob_score=True, random_state=None, verbose=0,
                       warm_start=False)

In [0]:
pred1= RFmodel.predict(X)
print(confusion_matrix(pred1, y))

[[282017      0]
 [     0     85]]


In [0]:
import pickle 
# Saving model to disk
pickle.dump(RFmodel, open('model.pkl','wb'))  #precompiled foremat model file

In [0]:
pred = RFmodel.predict(test)
pred = pd.DataFrame(pred)
sub_df=pd.read_csv('/content/drive/My Drive/Incident/Sample Submission.csv')
sub_df

,ID,prediction1
0,1,2 - Medium
1,2,2 - Medium
2,3,2 - Medium
3,4,2 - Medium
4,5,2 - Medium
...,...,...
42509,42510,3 - Low
42510,42511,3 - Low
42511,42512,3 - Low
42512,42513,3 - Low


In [0]:
pred[0].value_counts()

0.0    42482
1.0       32
Name: 0, dtype: int64

In [0]:
sub_df['prediction1'].value_counts()

2 - Medium    40301
3 - Low        2213
Name: prediction1, dtype: int64

In [0]:
datasets=pd.concat([sub_df['ID'],pred],axis=1)

datasets.columns=['Id','prediction1']
prediction2 = { 0.0:'2 - Medium',
                1.0:'3 - Low',
                2.0:'1 - High',
}
datasets['prediction1'] = datasets.prediction1.map(prediction2)
datasets.to_csv('/content/drive/My Drive/Incident/oversample_RF.csv',index=False)      #---score= 0.9478

In [0]:
datasets.prediction1.value_counts()

2 - Medium    42481
3 - Low          33
Name: prediction1, dtype: int64